In [53]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast

from sklearn.neighbors import NearestNeighbors

In [54]:
# get a user data (use mockup data untill there's user api)
user_tag_score_vector = np.array(MOCK_USER[1])

# get attractions data
attraction_tag_score_data = []
attraction_ref = []
try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/attraction/getAllData"
    res_all_attractions = requests.post(url=API_ENDPOINT).json()
    
    for cur_attraction in res_all_attractions['attractions']:
        cur_tag_score = cur_attraction['attractionTag']['attractionTagFields']
        cur_ref = (cur_attraction['_id'], cur_attraction['name'])

        attraction_tag_score_data.append(list(cur_tag_score.values()))
        attraction_ref.append(cur_ref)
        
except Exception as e:
    print("get all attraction data failed !")
    print(e)

for cur_tag, cur_ref in zip(attraction_tag_score_data, attraction_ref):
    print(cur_ref)
    print(cur_tag)



('67506bb4c31fe5f5de59a1e0', 'Sam Collection')
[5, 8, 10, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
('675926d58928ce599be6a6d8', 'time test 4')
[5, 8, 10, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
('675927ae8928ce599be6a6db', 'time test 4')
[5, 8, 10, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
('675929ea8928ce599be6a944', 'เลิฟ ไดวิ่ง ภูเก็ต')
[1, 0.8, 0, 0, 0, 0.2, 0.9, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0, 0]
('675929f18928ce599be6a946', 'เลิฟ ไดวิ่ง ภูเก็ต')
[1, 0.8, 0, 0, 0, 0.2, 0.9, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0, 0]
('675929f18928ce599be6a948', 'Andaman Sea Kayaks')
[0.9, 0.8, 0.1, 0, 0.1, 0.7, 0.9, 0, 0, 0.2, 0.2, 0, 0, 0, 0, 0, 0.1, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0.7, 0.5, 0.2, 0.

In [55]:
# train KNN Model
attraction_tag_score_matrix = np.array(attraction_tag_score_data.copy())
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(attraction_tag_score_matrix)

NearestNeighbors(metric='cosine')

In [56]:
# find top recommendations
user_tag_score_vector = user_tag_score_vector.reshape(1, -1)
distances, indices = knn.kneighbors(user_tag_score_vector)

# Print the recommendations
print("Top Recommendations:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {attraction_ref[idx][1]} (Similarity: {1 - distances[0][i]:.2f})")

Top Recommendations:
1. อาร์ทสเปซ แกลลอรี่ แอนด์มิวสิค บาร์ (Similarity: 1.00)
2. 75 Stang Art Studio and Reggae Bar (Similarity: 0.93)
3. Bless Art Gallery (Similarity: 0.88)
4. Beloved tattoo (Similarity: 0.88)
5. หอศิลป์กำพล (Similarity: 0.86)


In [57]:
distances

array([[0.        , 0.07130913, 0.11929114, 0.12453446, 0.13514656]])

In [58]:
indices

array([[ 98, 249, 340, 218, 241]])